In [1]:
import os

import sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
%load_ext tensorboard

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ex 12:
Implement a custom layer that performs Layer Normalization (we will use this  type of layer in Chapter 15):

In [2]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, epsilon=1e-3, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        
    def build(self, input_shape):
        self.alpha = self.add_weight(
                    name="alpha", shape=input_shape[-1:], dtype=tf.float32, 
                    initializer="ones")
        self.beta = self.add_weight(
                    name="beta", shape=input_shape[-1:], dtype=tf.float32, 
                    initializer="zeros")
        super().build(input_shape)
        
    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.epsilon)) + self.beta
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.epsilon}

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
X = X_train.astype(np.float32)

custom_layer_norm = LayerNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=3.80914e-08>

# Ex 13
Train a model using a custom training loop to tackle the Fashion MNIST dataset (see Chapter 10).

a. Display the epoch, iteration, mean training loss, and mean accuracy over each epoch (updated at each iteration), as well as the validation loss and accuracy at the end of each epoch

In [5]:
mnist = keras.datasets.mnist

(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

In [6]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [7]:
X_train.shape, y_train.shape

((55000, 28, 28), (55000,))

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [12]:
n_epochs = 100
batch_size = 128
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.SGD(lr=1e-3)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.Accuracy()]

In [16]:
from tqdm.notebook import trange
from collections import OrderedDict

with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        iterate = iter(dataset.batch(batch_size))
        with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
            X_b, y_b = next(iterate)
            with tf.GradientTape() as tape:
                y_pred = model(X_b)
                loss = tf.reduce_mean(loss_fn(y_b, y_pred))
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            for var in model.variables:
                if var.constraint is not None:
                    variable.assign(variable.constraint(variable))
            status = OrderedDict()
            mean_loss(loss)
            status["loss"] = mean_loss.result().numpy()
            for metric in metrics:
                metric(y_b, y_pred)
                status[metric.name] = metric.result().numpy()
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

KeyboardInterrupt: 

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [45]:
dataset

<TensorSliceDataset shapes: (28, 28), types: tf.float64>

In [82]:
d = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
a = dataset.batch(50000)
iterate = iter(a)

In [85]:
next(iterate)

StopIteration: 